# LLM Agent Capabilities
- List down places where `Harry Potter` movie franchise was shot in England.
- Verify if there is any Famous Cricket Ground nearby, if yes, identify the location as area of interest.
- Plot both locations on a map

## Planning
- What Tools would the agent need?
    - Search API
    - Web Scraping
    - Travel time calculator (custom tool)
    - Plotting tools (map)

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
# Setup langfuse client
from langfuse import Langfuse

langfuse = Langfuse(
	secret_key=os.getenv("SECRET_KEY"),
	public_key=os.getenv("PUBLIC_KEY"),
	host=os.getenv("LANGFUSE_HOST"),
)

In [ ]:
# authenticate langfuse client connection
from langfuse import get_client

langfuse = get_client()

# Verify connection does not use in production as this is a synchronous call
if langfuse.auth_check():
	print("Langfuse client is authenticated and ready!")
else:
	print("Authentication failed. Please check your credentials and host.")

In [ ]:
# setup llm observability
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

# set up opentelemetry tracer
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

# set up smolagents instrumentor
SmolagentsInstrumentor().instrument(trace_provider=trace_provider)

In [ ]:
# get rest of api key setup
hf_write_token = os.getenv("HF_WRITE_TOKEN")
openai_api_key = os.getenv("OPENAI_API_KEY")
serp_api_key = os.getenv("SERP_API_KEY")  # search api
serper_key = os.getenv("SERPER_KEY")  # search api

In [ ]:
import math
from typing import Optional, Tuple
from smolagents import tool

In [ ]:
# helper function to calculate distance between two points on earth
def to_radians(degrees: float) -> float:
	return degrees * math.pi / 180

In [ ]:
# Earth raidus in km
EARTH_RADIUS_KM = 6371.0

In [ ]:
# custom tool to identify travel time via chartered flight
@tool
def calculate_air_travel_time(
	origin_coords: Tuple[float, float],
	destination_coords: Tuple[float, float],
	cruising_speed_kmh: Optional[float] = 925,
) -> float:
	"""Calculate the air travel time between two points.
	Args:
	    origin_coords: Tuple of (latitude, longitude) for the origin point.
	    destination_coords: Tuple of (latitude, longitude) for the destination point.
	    cruising_speed_kmh: The cruising speed in km/h. Defaults to 925 for cross-atlantic chartered flights.
	Returns:
	    float: The air travel time in hours.
	"""
	# extract lat long from input args
	lat1, long1 = map(to_radians, origin_coords)
	lat2, long2 = map(to_radians, destination_coords)

	dlong = long2 - long1
	dlat = lat2 - lat1

	a = (
		math.sin(dlat / 2) ** 2
		+ math.cos(lat1) * math.cos(lat2) * math.sin(dlong / 2) ** 2
	)
	c = 2 * math.asin(math.sqrt(a))
	distance = EARTH_RADIUS_KM * c

	# add 5% for air traffic congestion
	final_distance = distance * 1.05

	# calculate air travel time in hours
	if final_distance and cruising_speed_kmh:
		air_travel_time_hours = (
			final_distance / cruising_speed_kmh + 1.0
		)  # takeoff and landing procedure
		return round(air_travel_time_hours, 2)

In [ ]:
# helper function to get lat long of a location
from geopy.geocoders import Nominatim


def get_lat_long(location_name: str) -> Optional[Tuple[float, float]]:
	"""Get the latitude and longitude of a location using the Nominatim API.
	Args:
	    location_name (str): The name of the location.
	Returns:
	    Optional[Tuple[float, float]]: A tuple containing the latitude and longitude, or None if the location could not be found.
	"""
	geolocator = Nominatim(user_agent="geopy_example", timeout=10)
	location = geolocator.geocode(location_name)

	if location:
		latitude, longitude = location.latitude, location.longitude
		return latitude, longitude
	else:
		return None

In [ ]:
print(f"Location of Ahmedabad is {get_lat_long('Ahmedabad, India')}")
print(f"Location of London is {get_lat_long('London, UK')}")

In [ ]:
# helper function to convert hours into days, hours, and minutes
def convert_hours(total_hours):
	"""
	Converts a given number of hours into days, remaining hours, and minutes.

	Args:
	    total_hours (float or int): The total number of hours to convert.

	Returns:
	    tuple: A tuple containing (days, hours, minutes).
	"""
	# Calculate days
	days = int(total_hours // 24)

	# Calculate remaining hours after extracting days
	remaining_hours = total_hours % 24

	# Calculate whole hours from the remaining_hours
	hours = int(remaining_hours)

	# Calculate minutes from the fractional part of remaining_hours
	minutes = int((remaining_hours - hours) * 60)

	return days, hours, minutes


# Example usage:
hours_input = 75.5
days, hours, minutes = convert_hours(hours_input)
print(
	f"{hours_input} hours is equivalent to {days} days, {hours} hours, and {minutes} minutes."
)

hours_input_2 = 48
days_2, hours_2, minutes_2 = convert_hours(hours_input_2)
print(
	f"{hours_input_2} hours is equivalent to {days_2} days, {hours_2} hours, and {minutes_2} minutes."
)

hours_input_3 = 1.75
days_3, hours_3, minutes_3 = convert_hours(hours_input_3)
print(
	f"{hours_input_3} hours is equivalent to {days_3} days, {hours_3} hours, and {minutes_3} minutes."
)

In [ ]:
# travel time between Ahmedabad and London
travel_time_hours = calculate_air_travel_time(
	(23.0215374, 72.5800568), (51.4893335, -0.1440551)
)
travel_time_print = convert_hours(travel_time_hours)

In [ ]:
print(
	f"Air Travel Time between Ahmedabad and London is {travel_time_print[0]} days, {travel_time_print[1]} hours, and {travel_time_print[2]} minutes."
)

### Now moving to agent creation

In [ ]:
from smolagents import (
	CodeAgent,
	GoogleSearchTool,
	InferenceClientModel,
	VisitWebpageTool,
)

In [ ]:
model = InferenceClientModel(
	model_id="Qwen/Qwen2.5-Coder-32B-Instruct",
	provider="together",
	api_key=hf_write_token,
)

In [ ]:
task = """find all harry potter filming locations in the world. calculate the time to travel to the location from Ahmedabad (located at 23.0225° N, 72.5714° E), India via passenger flight.
if there are any cricket grounds near the filming locations, list them out as well.  Return to me all these information in a nicely formatted pandas dataframe."""

In [ ]:
# Serp api key has to be part of environment, there is no option to pass it as argument
os.environ["SERPAPI_API_KEY"] = serp_api_key

In [ ]:
agent = CodeAgent(
	model=model,
	tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_air_travel_time],
	additional_authorized_imports=["pandas"],
	max_steps=20,
)

In [ ]:
result = agent.run(task.strip())

In [ ]:
import boto3

client = boto3.client(
	"bedrock-runtime",
	region_name="us-east-1",
	aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
	aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
)

In [ ]:
from smolagents import AmazonBedrockServerModel

bedrock_model = AmazonBedrockServerModel(
	model_id="anthropic.claude-3-haiku-20240307-v1:0",
	client=client,
)

In [ ]:
task = """find all harry potter filming locations in the world. calculate the time to travel to the location from Ahmedabad (located at 23.0225° N, 72.5714° E), India via passenger flight.
if there are any cricket grounds near the filming locations, list them out as well.  Return to me all these information in a nicely formatted pandas dataframe."""

In [ ]:
agent_bedrock = CodeAgent(
	model=bedrock_model,
	tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_air_travel_time],
	additional_authorized_imports=["pandas"],
	max_steps=20,
)

In [ ]:
agent_bedrock.run(task.strip())

In [ ]:
import boto3

client = boto3.client(
	"bedrock-runtime",
	# region_name="us-east-1",
	region_name="ap-south-1",
	aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
	aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
)

In [ ]:
from smolagents import AmazonBedrockServerModel

bedrock_model = AmazonBedrockServerModel(
	model_id="mistral.mistral-large-2402-v1:0",
	client=client,
)

In [ ]:
# You make comprehensive report after visiting many websites.
# Dont' hesitate to search for many queries at once in a for loop.
# For each data point that you find, visit the source URL to confirm numbers.
task = """
find all harry potter filming locations in the world, calculate the time to travel to the location from here (we are located at Ahmedabad, India 23.0225° N, 72.5714° E), and return them to me as a pandas dataframe.
Also, give me cricket stadiums if there are any nearby to the filming locations""".strip()

In [ ]:
agent_bedrock = CodeAgent(
	model=bedrock_model,
	tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_air_travel_time],
	additional_authorized_imports=["pandas"],
	max_steps=20,
)

In [ ]:
# agent_bedrock.planning_interval = 4
agent_bedrock.run(task.strip())